# Bug- small mass of elements created- numerical error?
When the moles of elements are converted to moles of basis species a small non-zero amount of every element seems to be created.

In [7]:
import pyDEW
import numpy as np
from scipy.optimize import root_scalar
import os
import matplotlib.pyplot as plt

Copied from DEWFluid_module.py:

In [230]:
class Fluid(pyDEW.fluid.fluid):
    def __init__(self, system, t, p, n, fO2, max_iterations=99, aH2O_mode='unity'):
        self.system = system
        self.T = t
        self.P = p
        self.pH = 5.0
        self.fO2 = fO2
        self.molalities, self.k = self._calc_basis_species_molalities(n)
        self.mineral_eq = {}
        self.solid_solutions = {}
        self.uebal = 'H+'
        self.uacion = ''
        self.nxmods = []
        self.dummy_temperature = 500.0
        if max_iterations > 99:
            raise core.InputError("EQ3 only supports a maximum of 99 iterations.")
        elif max_iterations < 0 or isinstance(max_iterations, int) is False:
            raise core.InputError("Maximum iterations must be a positive integer.")
        self.max_iterations = max_iterations
        
        if aH2O_mode not in ['unity', 'molfraction']:
            raise core.InputError("aH2O mode note recognised.")
        self.aH2O_mode = aH2O_mode

        eqpt_working_directory = 'eq_working'
        eq3_working_directory = 'eq_working'
        eqpt_executable_name = 'EQPT'
        eq3_executable_name = 'EQ3'
        input_filename = 'input'

        # If O2 is being set by mineral equilibrium:
        if 'O2(G)' in self.mineral_eq:
            self.fO2 = None

        # Create DATA0 and run EQPT
        # Determine filename automatically
        # if data0_filename is None and core.operatingsystem == 'Darwin':
        data0_filename = 'DATA0'
        # elif data0_filename is None and core.operatingsystem == 'Linux':
        #     data0_filename = 'data0'
        # Create the working directory if it doesn't exist:
        if not os.path.isdir(eqpt_working_directory):
            os.makedirs(eqpt_working_directory)
        # Create DATA0
        self.system.make_data0(self.T, self.P, format='pyDEW',
                               filepath=eqpt_working_directory + '/' + data0_filename,
                               dummy_temperature=self.dummy_temperature - 50)
        # Run EQPT
        pyDEW.core.run_eqpt(working_directory=eqpt_working_directory,
                           executable_name=eqpt_executable_name)

        # Create the eq3 working directory if it doesn't exist:
        if not os.path.isdir(eq3_working_directory):
            os.makedirs(eq3_working_directory)

        # Check to see if the working directories are different:
        if eq3_working_directory != eqpt_working_directory:
            os.system("cp " + eqpt_working_directory + '/data1 '
                      + eq3_working_directory + '/data1')

        # Create input file and run EQ3
        self._make_input(filepath=eq3_working_directory
                         + '/' + input_filename, format='pyDEW')
        pyDEW.core.run_eq3(working_directory=eq3_working_directory,
                          executable_name=eq3_executable_name)

        # Collect output
        self.eq3output = pyDEW.output.eq3output(
            filepath=eq3_working_directory + '/output')
        self.elemental_comp = self.eq3output.elemental_comp.set_index(
            'element').astype('float')
        # self.elemental_comp_ppm = dict(self.elemental_comp.ppm)
        self.elemental_comp_molality = dict(self.elemental_comp.molality)
        # self.pH = float(self.eq3output.electrochemistry['pH'][0])
        self.aqueous_species = self.eq3output.aqueous_species
        # self.fO2 = float(self.eq3output.redox['log_fO2'][0])
        # self.mineral_saturation = self.eq3output.mineral_saturation

    def _calc_basis_species_molalities(self, mols):
        n = np.zeros(self.system.n_elements+1)
        n[:-1] = mols
        x = np.linalg.solve(self.system._basis_species_matrix.T, n)
        # EQ3 performs calculations assuming 1 kg of solvent, k will only be the correct scaling
        # if there is no additional H2O in complexes. May have to find k by iteration.
        k = 55.5086815578/x[0]
        x = x*k
        molalities = {}
        for i, species in zip(range(len(self.system.basis_species)),
                              self.system.basis_species_names):
            if species != 'H+':
                molalities[species] = x[i]
        return molalities, k

    def _gibbs_energy(self):
        g = 0
        g += 55.5086815578 * \
            self.system.species['H2O'].gibbs_energy(self.T, self.T)/self.k
        for i, row in self.aqueous_species.iterrows():
            if row.species != 'H+':
                g += (self.system.species[row.species].gibbs_energy(self.T, self.P)
                      + 8.314*self.T*np.log(row.activity))*row.molality/self.k
            else:
                g += (8.314*self.T*np.log(row.activity))*row.molality/self.k
        return g


    def _element_projection(self, mols, total_mols_h):
        n = np.zeros(self.system.n_elements+1)
        n[:-1] = mols
        x = np.linalg.solve(self.system._basis_species_matrix.T, n)
        # x[0] is H2O
        print(x)
        k = total_mols_h/x[0]*2
        x = x*k
        molalities = {}
        for i, species in zip(range(len(self.system.basis_species)),
                              self.system.basis_species_names):
            if species not in ['H+', 'H2O', 'O2(G)'] and x[i] > 0:
                molalities[species] = x[i]
        return molalities, k

    def _mass_misfit(self, total_mols_h, mols):
        self.molalities, self.k = self._element_projection(mols, total_mols_h)

        # Create input file and run EQ3
        self._make_input(filepath=self.eq3_working_directory
                         + '/' + self.input_filename, format='pyDEW')
        pyDEW.core.run_eq3(working_directory=self.eq3_working_directory,
                          executable_name=self.eq3_executable_name)

        # Collect output
        self.eq3output = pyDEW.output.eq3output(
            filepath=self.eq3_working_directory + '/output')
        self.elemental_comp = self.eq3output.elemental_comp.set_index(
            'element').astype('float')

        calc_h = self.elemental_comp.molality['H']
        return calc_h - total_mols_h

Use a small demo system:

In [231]:
elements = ['O', 'H', 'Si', 'Na', 'Cl', 'K']
basis_species = ['H2O', 'H+', 'H4SIO4(AQ)', 'NA+', 'CL-', 'K+', 'O2(G)']
other_species = ['H6SI2O7(AQ)', 'H8SI3O10(AQ)', 'H3SIO4-', 'OH-', 'O2(AQ)', 'NACL(AQ)']

In [232]:
system = pyDEW.System(basis_species=basis_species,
                     other_species=other_species,
                     elements=elements,
                     minerals=[],
                     gases=[],
                     hydrated_species={},
                     solid_solutions={})

## Recreate bug:

In [233]:
moles = np.array([0.3 + 0.03333, 
                  0.6 + 0.0, 
                  0.0,
                  0.06666,
                  0.0,
                  0.0
                 ])

In [234]:
fl = Fluid(system, 723.15, 10000.0, moles, -12.0)

In [235]:
fl.molalities

{'H2O': 55.5086815578,
 'H4SIO4(AQ)': 0.0,
 'NA+': 11.100737145300297,
 'CL-': 0.0,
 'K+': 0.0,
 'O2(G)': 2.3110362228143043e-15}

In [236]:
fl.aqueous_species

,species,molality,log_g,activity
0,OH-,1.110100e+01,-0.4094,4.325000e+00
1,NA+,1.110100e+01,-0.4094,4.325000e+00
2,H+,5.097733e-09,-0.4094,1.986000e-09
3,O2(AQ),8.562488e-17,0.0000,8.562000e-17


In [227]:
fl.elemental_comp

,ppm,molality
element,,
O,1.065715e+06,66.609682
H,1.230831e+05,122.118363
NA,2.552094e+05,11.101000


In [228]:
fl.k

166.5277099504995

In [229]:
fl.elemental_comp/fl.k

,ppm,molality
element,,
O,6399.625260,0.399992
H,739.114819,0.733322
NA,1532.534356,0.066662
